In [1]:
# import pytorch
import torch

# define a tensor
torch.FloatTensor([2])
torch.cuda.current_device()
torch.cuda.device(0)
 torch.cuda.device_count()
    torch.cuda.get_device_name(0)

tensor([2.])

In [1]:
import logly
from random import randint

In [2]:
val_flag = True
epoch_size = 5
batch_size = 10
batch_log = logly.logly(val_flag, epoch_size, batch_size)

first: 2, second: 5, third:10


In [3]:
for phase in ['train', 'val']:
    for epoch in range(epoch_size):
        for batch in range(batch_size):
            loss = randint(0, 100)
            batch_log.log(phase, epoch, batch, loss)

In [4]:
batch_log.write("data/logs/", "test")





In [23]:
pot = potly()
output_dir = './data/outputs/plots/'
filename = 'test' + str(1)
N = 100
x = np.random.rand(N)
y = np.random.rand(N)

pot.save_plot(x,y,output_dir, filename)